In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [3]:
S0 = df['Adj_S0'].unique()

In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [6]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(N_t0, batch_size, T))
    eps = 100
    for i in range(N_t0):
        for t in range(T):
            data[i,:,t] = (S0[i]-eps) + 2*eps*data[i,:,t]
    return data

In [7]:
BATCH_SIZE = 1000
S = margDistr(BATCH_SIZE)

# different S for different t0: 13*Batch_size*2

In [8]:
Phi = (S[:,:,1]-S[:,:,0])
# define the payoff function of the derivative

### Variables

In [9]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [10]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [11]:
# parameter: Delta_0
Delta_0 = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'Delta0'
)

In [12]:
# neural network: Delta_1(S_1)
Delta_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [36]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [37]:
# take a look
hedgePrice

[<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-6389.15646054])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-3698.5242348])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2269.13340404])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-3147.19360047])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-3920.59495079])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-6240.36056683])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-4342.9442266])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-8688.08154928])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-7078.82289915])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-7348.22823644])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-13198.93184426])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-14797.57062434])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-8154.18459272])>]

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{0}\cdot(S_1-S_0)+\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
# one more variable Delta_0 Delta_0(S1-S0)

In [15]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
    p += Delta_0[n,0]*(S[n,:,0]-S0[n])
    p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
    hedgeTerm.append(p)

In [16]:
# take a look
hedgeTerm

[<tf.Tensor: shape=(1000,), dtype=float64, numpy=
 array([-2.00528716e+04, -5.55895169e+03,  9.78382602e+03,  4.40939713e+03,
        -1.73181090e+04,  1.15981078e+04, -2.05386255e+04,  1.45558009e+03,
         1.09830101e+04, -1.83922175e+04, -2.13643141e+04,  1.37823510e+04,
         1.16330375e+04,  6.05887075e+03,  1.43337605e+04,  1.01883905e+04,
        -1.21116512e+04,  5.87160357e+03,  1.13534761e+03, -4.83729439e+03,
        -1.04436209e+04,  1.02351370e+04, -1.87744608e+04,  1.21697304e+04,
        -1.85239255e+04,  1.02171650e+04, -4.40328692e+03, -1.20959151e+04,
        -1.87134243e+04,  1.62645297e+04,  2.34055817e+04, -3.05011701e+04,
         7.56761683e+03, -2.76021975e+04,  2.93025750e+03,  2.10293984e+03,
         1.92062986e+04,  1.86022118e+04, -1.70449729e+04, -1.75631637e+03,
        -7.14253764e+03,  1.07234757e+04,  2.79657779e+04,  3.76715248e+02,
        -2.83381122e+04, -6.44519420e+03, -9.54337685e+03,  5.98555586e+03,
         7.92477230e+03, -1.75714011e+

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{0}\cdot(S_1-S_0)-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [17]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [18]:
# take a look
Price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 144.07691869,  254.50477277,  352.89063544,  518.76899753,
        998.4125195 , -166.91547701,  352.23413875, -420.62839805,
        181.10438401,  782.74699473,   65.19684025,  -10.71324262,
        350.51021804])>

In [19]:
# take a look
Penalty

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([1.20986647e+08, 7.18039029e+07, 4.87633502e+08, 1.51771556e+09,
       3.44316490e+09, 1.33495092e+08, 1.01766188e+09, 3.49198701e+08,
       1.12085451e+08, 3.40800158e+07, 5.02763426e+09, 3.14520917e+08,
       2.40800788e+07])>

In [20]:
# take a look
ObjFunc

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([1.20986648e+10, 7.18039055e+09, 4.87633505e+10, 1.51771556e+11,
       3.44316491e+11, 1.33495091e+10, 1.01766188e+11, 3.49198697e+10,
       1.12085453e+10, 3.40800236e+09, 5.02763426e+11, 3.14520917e+10,
       2.40800823e+09])>

### Optimisation

In [21]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
        p += Delta_0[n,0]*(S[n,:,0]-S0[n])
        p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [22]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [23]:
nIter = 10000
threshold = 1e-3
prev_price = tf.ones((N_t0,), dtype = tf.float64) * float('inf')

for step in range(nIter):
    if (step+1) % 500 == 0:
        print("Processing progress: %d / %d" %(step+1, nIter))
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        tape.watch(Delta_0)
        for variable in Delta_1.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S)
    gradients_d, gradients_Lambda, gradients_Delta_0 = tape.gradient(obj, [d, Lambda, Delta_0])
    gradients_Delta_1 = tape.gradient(obj, Delta_1.trainable_variables)
    
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda, gradients_Delta_0] + gradients_Delta_1, 
            [d, Lambda, Delta_0] + Delta_1.trainable_variables)
    )
    
    diff = tf.abs(price - prev_price)
    if tf.reduce_max(diff) < threshold:
        print("Converged at step", step+1)
        break
    
    prev_price = price
    
    del tape

Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000
Processing progress: 3000 / 10000
Processing progress: 3500 / 10000
Processing progress: 4000 / 10000
Processing progress: 4500 / 10000
Processing progress: 5000 / 10000
Processing progress: 5500 / 10000
Processing progress: 6000 / 10000
Processing progress: 6500 / 10000
Processing progress: 7000 / 10000
Processing progress: 7500 / 10000
Processing progress: 8000 / 10000
Processing progress: 8500 / 10000
Processing progress: 9000 / 10000
Processing progress: 9500 / 10000
Processing progress: 10000 / 10000


In [24]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ -6388.43792764,  -3698.07372487,  -2268.80667906,  -3146.77969252,
        -3920.0507071 ,  -6239.71346609,  -4342.4290768 ,  -8687.20103548,
        -7078.03322473,  -7347.37104726, -13197.58757599, -14796.07170401,
        -8153.31964082])>

In [25]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ -4558.90712293,  -3698.07372487,  -2268.80667906,  -3146.77969252,
        -3920.0507071 ,  -6237.71360846,  -4342.4290768 ,  -8685.00700901,
        -7078.03322473,  -7347.37104726, -13197.58757599, -14779.58808827,
        -8153.31964082])>

In [35]:
obj-price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([1829.53080471,    0.        ,    0.        ,    0.        ,
          0.        ,    1.99985763,    0.        ,    2.19402647,
          0.        ,    0.        ,    0.        ,   16.48361574,
          0.        ])>